In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Dados do Problema

In [2]:
J = 4
M = 2

TP = [18, 14, 12, 6]
TC = [4, 4, 6, 6]
st = [2, 4];

# Funções auxiliares

Função objetivo, recebe uma solução e calcula o tempo de enfesto. O tempo é calculado somando os tempos médios de cada máquina junto ao tempo de setup multiplicado pelas ordens de cada máquina.

In [3]:
def FO(Sol,TP,TC,st):
    """ Recebe uma solução Sol, os tempos de enfesto TP e setup st. 
        Retorna o tempo que demora para terminar o enfesto das ordens de produção"""
    
    #tempos de setup por máquina
    s1 = st[0]
    s2 = st[1]
    
    Time_enf = np.zeros(len(TP))
    Time_cutt = np.zeros(len(TC))
    # tempo de enfesto máquina 1
    t1 = 0
    for i in Sol[0]:
        Time_enf[i] = t1 + TP[i] + s1
        t1 += TP[i] + s1
    t2 = 0
    for i in Sol[1]:
        if t2>Time_enf[i]:
            Time_cutt[i] = t2 + TC[i] + s2
            t2 += TC[i] + s2
        else:
            Time_cutt[i] = Time_enf[i] + TC[i] + s2
            t2 += (Time_enf[i]-t2) + TC[i] + s2
    return Sol,max(t1,t2),Time_enf,Time_cutt

Função Random_start, gera uma solução aleatória

In [4]:
def Random_start(J,M):
    m1 = np.arange(0,J,1)
    m2 = np.arange(0,J,1)

    np.random.shuffle(m1)
    np.random.shuffle(m2)
    
    return [m1,m2]

# Operadores

Operador1 escolhe de forma aleatória uma ordem e troca de lugar aleatóriamente

In [5]:
def operator1(Sol,TP,TC):
    backtrack=0
    if np.random.uniform()>0.5:
        n = len(TP)
        v = list(Sol[0])
        backtrack = 0
    else:
        n = len(TC)
        v = list(Sol[1])
        backtrack = 1
    #Escolhe 1 orden para trocar de lugar
    Swap = int(n*np.random.uniform())
    v.remove(Swap)
    #escolhe uma posicao aleatoria nova
    pos = np.random.randint(0,len(v)+1)
    v = v[:pos]+[Swap]+v[pos:]
    newSol = Sol.copy()
    newSol[backtrack] = v
    return newSol

Operador2 escolhe 2 ordens e troca o lugar de uma pela outra

In [6]:
def operator2(Sol,TP,TC):
    n = len(TP)
    backtrack = 0
    #Escolhe 2 ordens para trocar de lugar
    if np.random.uniform()>0.5:
        v = list(Sol[0])
        backtrack = 0
    else:
        v = list(Sol[1])
        backtrack = 1
    choices = np.arange(0,n,1)
    Swap1 = np.random.choice(choices,replace=False)
    Swap2 = np.random.choice(choices,replace=False)
    O1 = v[Swap1]
    O2 = v[Swap2]

    v[Swap1] = O2
    v[Swap2] = O1
    
    newSol = Sol.copy()
    newSol[backtrack] = v
    return newSol

Função random_neigh realiza uma busca local, com probabilidade p1 escolhe uma ordem e realiza uma troca de forma gulosa, com probabilidade 1-p1 aplica os operadores 1 e 2.

In [7]:
def random_neigh(Sol,TP,TC,st, FO):
    p1 = np.random.uniform()
    p2 = np.random.uniform()
    n = len(TP)
    backtrack = 0
    if p1>2/6:
            n = len(TP)
            #Escolhe 2 ordens para trocar de lugar
            if np.random.uniform()>0.5:
                v = list(Sol[0])
                backtrack = 0
            else:
                v = list(Sol[1])
                backtrack = 1
            c1 = int(n*np.random.uniform())
            indices = [i for i in range(len(v)) if v[i] != c1]
            v.remove(c1)
            s2 = v.copy()
            best = np.inf
            for i in indices:
                s2 = v.copy()
                s2.insert(i,c1)
                c = Sol.copy()
                c[backtrack] = s2
                _,price,_,_ = FO(c,TP,TC,st)
                if best>price:
                    best = price
                    sol_best = c
            v = sol_best
    else:
        
        v = operator1(Sol,TP,TC)
        if p2<0.8:
            v = operator2(Sol,TP,TC)
    return v

Simulated annealing

In [8]:
def sa(TP, TC, st, LS, td, T0, L=10, maxiter=100):
    S0 = Random_start(len(TP),len(st))
    Best_sol =[]
    S_best,F_best,_,_ = FO(S0,TP,TC,st)
    S = S0
    cF= F_best
    Best_sol.append(F_best)
    T = T0
    niter = 0
    for iteration in range(maxiter):
        temploop = 0
        improv = 0
        while temploop<L and improv<np.ceil(L/100):
            # Gera um vizinho aleatório
            nS = LS(S, TP,TC,st, FO)

            # Calcula as funções de custo para o vizinho
            _,nF,_,_ = FO(nS,TP,TC,st)
            diff = cF-nF

            # Verifica se o vizinho é melhor ou se deve ser aceito de acordo com a probabilidade
            if nF < cF or (np.exp(-diff/T) > np.random.uniform()):
                S = nS

            # Verifica se a solução atual é a melhor até o momento
            if nF < F_best:
                F_best = nF
                S_best = nS
                Best_sol.append(F_best)
                improv += 1
            temploop +=1
        # Reduz a temperatura
        T = td(T0,iteration)

        
    return S_best,F_best,Best_sol

# Dados de temperatura

In [9]:
TF = 0.001
T0 = 5000
alpha = 0.99

Função td, realiza o decaimento de temperatura exponencial

In [10]:
def td(T0,i):
    return T0*alpha**i

In [11]:
sa(TP, TC, st, random_neigh, td, T0, L=10, maxiter=100)

([[3, 1, 2, 0], array([1, 2, 3, 0])], 66.0, [86.0, 66.0])